In [59]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import csv_loader,CSVLoader
from langchain_community.vectorstores import Chroma,FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pandas as pd
import os
from dotenv import load_dotenv

In [60]:
from langchain.chains import RetrievalQA,ConversationalRetrievalChain

In [61]:
load_dotenv()

True

In [62]:
loader =CSVLoader("pocdata.csv")
doc=loader.load()
doc

[Document(metadata={'source': 'pocdata.csv', 'row': 0}, page_content='Attributes: Config 1\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 1.0\nBusiness Attendent: 1.0\nMonthly Recurring Charges: 10.0\nNet Recurring Charges: 100.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 1}, page_content='Attributes: Config 2\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 0.0\nBusiness Attendent: 0.0\nMonthly Recurring Charges: 10.0\nNet Recurring Charges: 100.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 2}, page_content='Attributes: Config 3\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 1.0\nBusiness Attendent: 0.0\nMonthly Recurring Charges: 15.0\nNet Recurring Charges: 125.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 3}, page_content='Attributes: Config 4\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 0.0\nBusiness Attendent: 1.

In [63]:
text_splitter=CharacterTextSplitter(chunk_size=500, 
                                    chunk_overlap=0)
texts=text_splitter.split_documents(doc)

In [64]:
texts

[Document(metadata={'source': 'pocdata.csv', 'row': 0}, page_content='Attributes: Config 1\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 1.0\nBusiness Attendent: 1.0\nMonthly Recurring Charges: 10.0\nNet Recurring Charges: 100.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 1}, page_content='Attributes: Config 2\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 0.0\nBusiness Attendent: 0.0\nMonthly Recurring Charges: 10.0\nNet Recurring Charges: 100.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 2}, page_content='Attributes: Config 3\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 1.0\nBusiness Attendent: 0.0\nMonthly Recurring Charges: 15.0\nNet Recurring Charges: 125.0'),
 Document(metadata={'source': 'pocdata.csv', 'row': 3}, page_content='Attributes: Config 4\nContract Term: 1 Month\nNo of Routers: 1.0\nNo of Telephones: 8.0\nBOT Service: 0.0\nBusiness Attendent: 1.

In [65]:
llm =ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [66]:
embeddings=OpenAIEmbeddings()

In [67]:
# from pinecone import Pinecone
# Pinecone.init(api_key=os.getenv("PineconeAPI_KEY"),environment="gcp-starter")

In [68]:
vectordb =Chroma.from_documents(texts,embedding=embeddings,persist_directory="C:/prpwork/01AT&TPOC/data")

In [69]:
vectordb.persist()

In [70]:
retrivers =vectordb.as_retriever(search={'k':7})

In [58]:
retrivers.("2 no of telephone")

TypeError: 'VectorStoreRetriever' object is not callable

In [37]:
from langchain.prompts import PromptTemplate

template = """Answer the question in your own words as truthfully as possible from the context given to you.
If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question".
If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"
Context: {context}

Human: {question}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["context", "question"], template=template
)

# Create the custom chain
if llm is not None and vectordb is not None:
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectordb.as_retriever(),
        return_source_documents=True,
        combine_docs_chain_kwargs={'prompt': prompt})

In [38]:
if llm is not None and vectordb is not None:
    chainqa = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectordb.as_retriever(),
        return_source_documents=True)

In [56]:
def query_llm(retriever, query):
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY")),
        retriever=retriever,
        return_source_documents=True,
    )
    result = qa_chain({'question': query, 'chat_history': "Telecom plan for 24 month"})
    result = result['answer']
    # st.session_state.messages.append((query, result))
    return result

In [57]:
res=query_llm(retrivers,"Tell me about telephone and broadban plan for 24 month")

ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Telecom plan for 24 month 